# Preliminary analysis of variables

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sweetviz as sv

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [85]:
df = pd.read_csv("data/fraud_oracle.csv")
print(df.head())

  Month  WeekOfMonth  DayOfWeek    Make AccidentArea DayOfWeekClaimed  \
0   Dec            5  Wednesday   Honda        Urban          Tuesday   
1   Jan            3  Wednesday   Honda        Urban           Monday   
2   Oct            5     Friday   Honda        Urban         Thursday   
3   Jun            2   Saturday  Toyota        Rural           Friday   
4   Jan            5     Monday   Honda        Urban          Tuesday   

  MonthClaimed  WeekOfMonthClaimed     Sex MaritalStatus  ...  AgeOfVehicle  \
0          Jan                   1  Female        Single  ...       3 years   
1          Jan                   4    Male        Single  ...       6 years   
2          Nov                   2    Male       Married  ...       7 years   
3          Jul                   1    Male       Married  ...   more than 7   
4          Feb                   2  Female        Single  ...       5 years   

  AgeOfPolicyHolder PoliceReportFiled WitnessPresent AgentType  \
0          26 to 30 

Discrepancy between the variables Age and AgeOfPolicyHolder. Chose to keep Age variable for interpretability within the model and ease when generating fraudulent cases.

In [54]:
df.columns

Index(['Month', 'WeekOfMonth', 'DayOfWeek', 'Make', 'AccidentArea',
       'DayOfWeekClaimed', 'MonthClaimed', 'WeekOfMonthClaimed', 'Sex',
       'MaritalStatus', 'Age', 'Fault', 'PolicyType', 'VehicleCategory',
       'VehiclePrice', 'FraudFound_P', 'PolicyNumber', 'RepNumber',
       'Deductible', 'DriverRating', 'Days_Policy_Accident',
       'Days_Policy_Claim', 'PastNumberOfClaims', 'AgeOfVehicle',
       'AgeOfPolicyHolder', 'PoliceReportFiled', 'WitnessPresent', 'AgentType',
       'NumberOfSuppliments', 'AddressChange_Claim', 'NumberOfCars', 'Year',
       'BasePolicy'],
      dtype='object')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15420 entries, 0 to 15419
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Month                 15420 non-null  object
 1   WeekOfMonth           15420 non-null  int64 
 2   DayOfWeek             15420 non-null  object
 3   Make                  15420 non-null  object
 4   AccidentArea          15420 non-null  object
 5   DayOfWeekClaimed      15420 non-null  object
 6   MonthClaimed          15420 non-null  object
 7   WeekOfMonthClaimed    15420 non-null  int64 
 8   Sex                   15420 non-null  object
 9   MaritalStatus         15420 non-null  object
 10  Age                   15420 non-null  int64 
 11  Fault                 15420 non-null  object
 12  PolicyType            15420 non-null  object
 13  VehicleCategory       15420 non-null  object
 14  VehiclePrice          15420 non-null  object
 15  FraudFound_P          15420 non-null

Seems that none of the variables have any missing values. No treatment of removal of NaN values necessary.

In [56]:
# Sweetviz report of the complete dataset
eda_report = sv.analyze(df, target_feat='FraudFound_P')
eda_report.show_html('reports/vehicle_fraud_EDA.html')

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report reports/vehicle_fraud_EDA.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Investigation of Age and AgeOfPolicyHolder variables, are they similar in any way? Can I drop one of them to avoid multicollinearity later on in training the model?

In [57]:
df[["Age", "AgeOfVehicle", "AgeOfPolicyHolder"]].head()

,Age,AgeOfVehicle,AgeOfPolicyHolder
0,21,3 years,26 to 30
1,34,6 years,31 to 35
2,47,7 years,41 to 50
3,65,more than 7,51 to 65
4,27,5 years,31 to 35


In [58]:
sorted(df["Age"].unique())[:10]

[0, 16, 17, 18, 19, 20, 21, 22, 23, 24]

In [59]:
sorted(df["AgeOfPolicyHolder"].unique())[:5]

['16 to 17', '18 to 20', '21 to 25', '26 to 30', '31 to 35']

In [60]:
age0df = df[df["Age"]==0]
age0df["AgeOfPolicyHolder"].unique()

array(['16 to 17'], dtype=object)

Seems that variable Age = 0 means that it is encoded in the 16 to 17 value of the AgeOfPolicyHolder variable. We will keep the AgeOfPolicyHolder variable since having categories is easier for model to interpret than individual ages.

This shows an imbalanced dataset of fraud, less than 10% of cases are detected as fraud.
Next we clean the dataset by removing variables that are irrelevant in a case of fraud.

In [86]:
df_cleaned = df.drop(columns=['PolicyNumber', 'RepNumber', 'Year', 'Month', 
                 'WeekOfMonth', 'DayOfWeek', 'Age'])

Encoding the categorical variables is next

In [87]:
for col in df_cleaned.select_dtypes(include=['object']).columns:
    print(f"{col} — unique values:")
    print(df_cleaned[col].unique())

Make — unique values:
['Honda' 'Toyota' 'Ford' 'Mazda' 'Chevrolet' 'Pontiac' 'Accura' 'Dodge'
 'Mercury' 'Jaguar' 'Nisson' 'VW' 'Saab' 'Saturn' 'Porche' 'BMW' 'Mecedes'
 'Ferrari' 'Lexus']
AccidentArea — unique values:
['Urban' 'Rural']
DayOfWeekClaimed — unique values:
['Tuesday' 'Monday' 'Thursday' 'Friday' 'Wednesday' 'Saturday' 'Sunday'
 '0']
MonthClaimed — unique values:
['Jan' 'Nov' 'Jul' 'Feb' 'Mar' 'Dec' 'Apr' 'Aug' 'May' 'Jun' 'Sep' 'Oct'
 '0']
Sex — unique values:
['Female' 'Male']
MaritalStatus — unique values:
['Single' 'Married' 'Widow' 'Divorced']
Fault — unique values:
['Policy Holder' 'Third Party']
PolicyType — unique values:
['Sport - Liability' 'Sport - Collision' 'Sedan - Liability'
 'Utility - All Perils' 'Sedan - All Perils' 'Sedan - Collision'
 'Utility - Collision' 'Utility - Liability' 'Sport - All Perils']
VehicleCategory — unique values:
['Sport' 'Utility' 'Sedan']
VehiclePrice — unique values:
['more than 69000' '20000 to 29000' '30000 to 39000' 'less than 2

It seems like there are 3 variables that could potentially cause multicollinearity in the future when training the model. PolicyType is a combination of VehicleCategory, and BasePolicy. It would be better to remove PolicyType if there is a direct one-to-one link between these variables. 

In [88]:
exp_policytype = df_cleaned['VehicleCategory'].astype(str) + " - " + df['BasePolicy'].astype(str)
df_cleaned['is_match'] = (df_cleaned['PolicyType'] == exp_policytype)

In [89]:
mismatches = df_cleaned[df_cleaned['is_match'] == False]

print(f"Number of mismatches: {len(mismatches)} out of {len(df_cleaned)}")
mismatches[['PolicyType', 'VehicleCategory', 'BasePolicy']].head()
df_cleaned.drop(columns=["is_match"], inplace=True)

Number of mismatches: 4987 out of 15420


It seems like about 32.3% of the dataset has a mismatch between the three categories, which means it is not redundant or repeating information. The VehicleCategory is the actual vehicle whereas the PolicyType is based on what was declared to the insurance company. If there is a mismatch between them, especially for vehicle category, it could raise a flag about potential fraudulent entries. If it's not an attempt at fraud, it could be a misreport of the vehicle, the policy info may be outdated or simply incorrect, or it could be due to a mistake in the data entry. We will create a flag variable for this mismatch, as it could potentially be due to a fraudulent attempt.

In [91]:
df_cleaned['ActualPolicyVehicle'] = df_cleaned['PolicyType'].str.split(' - ').str[0]
df_cleaned['ActualPolicyCoverage'] = df_cleaned['PolicyType'].str.split(' - ').str[1]

df_cleaned['PolicyMismatchFlag'] = (df_cleaned['ActualPolicyVehicle'] != df_cleaned['VehicleCategory']).astype(int)

In [92]:
print(df_cleaned['PolicyMismatchFlag'].value_counts())

0    10433
1     4987
Name: PolicyMismatchFlag, dtype: int64


In [93]:
df_cleaned.drop(columns=["ActualPolicyVehicle", "ActualPolicyCoverage"], inplace=True)

To encode into 0 and 1: 
- AccidentArea 
- Sex
- Fault 
- PoliceReportFiled 
- WitnessPresent 
- AgentType 

In [95]:
le = LabelEncoder()
df_cleaned['Sex'] = le.fit_transform(df_cleaned['Sex']) # 1:male, 0:female
df_cleaned['AccidentArea'] = le.fit_transform(df_cleaned['AccidentArea']) # 1:urban, 0:rural
df_cleaned['Fault'] = le.fit_transform(df_cleaned['Fault']) # 1: third party, 0: policy holder
df_cleaned['PoliceReportFiled'] = le.fit_transform(df_cleaned['PoliceReportFiled']) # 1: yes, 0: no
df_cleaned['WitnessPresent'] = le.fit_transform(df_cleaned['WitnessPresent']) # 1: yes, 0: no
df_cleaned['AgentType'] = le.fit_transform(df_cleaned['AgentType']) # 1: internal, 0: external

In [69]:
df_cleaned.head()

,Make,AccidentArea,DayOfWeekClaimed,MonthClaimed,WeekOfMonthClaimed,Sex,MaritalStatus,Fault,PolicyType,VehicleCategory,...,AgeOfVehicle,AgeOfPolicyHolder,PoliceReportFiled,WitnessPresent,AgentType,NumberOfSuppliments,AddressChange_Claim,NumberOfCars,BasePolicy,PolicyMismatchFlag
0,Honda,1,Tuesday,Jan,1,0,Single,0,Sport - Liability,Sport,...,3 years,26 to 30,0,0,0,none,1 year,3 to 4,Liability,0
1,Honda,1,Monday,Jan,4,1,Single,0,Sport - Collision,Sport,...,6 years,31 to 35,1,0,0,none,no change,1 vehicle,Collision,0
2,Honda,1,Thursday,Nov,2,1,Married,0,Sport - Collision,Sport,...,7 years,41 to 50,0,0,0,none,no change,1 vehicle,Collision,0
3,Toyota,0,Friday,Jul,1,1,Married,1,Sedan - Liability,Sport,...,more than 7,51 to 65,1,0,0,more than 5,no change,1 vehicle,Liability,1
4,Honda,1,Tuesday,Feb,2,0,Single,1,Sport - Collision,Sport,...,5 years,31 to 35,0,0,0,none,no change,1 vehicle,Collision,0


Analyzing skewness of FraudFound_P and Deductible variables. Only 6% of entries are considered fraudulent. 96% of entries have a deductible of 400. 

In [96]:
print(df_cleaned['FraudFound_P'].value_counts())
df_cleaned['FraudFound_P'].value_counts(normalize=True)

0    14497
1      923
Name: FraudFound_P, dtype: int64


0    0.940143
1    0.059857
Name: FraudFound_P, dtype: float64

In [97]:
print(df_cleaned['Deductible'].value_counts())
df_cleaned['Deductible'].value_counts(normalize=True)

400    14838
700      311
500      263
300        8
Name: Deductible, dtype: int64


400    0.962257
700    0.020169
500    0.017056
300    0.000519
Name: Deductible, dtype: float64

Scaling the numerical variables using Min Max Scaling 

In [98]:
num_cols = df_cleaned.select_dtypes(include=['int64','float64']).columns
scaler = MinMaxScaler()
df_cleaned[num_cols] = scaler.fit_transform(df_cleaned[num_cols])

Separating the dataset cleaned into non fraudulent set and fraudulent set

In [99]:
non_fraudulent_entries = df_cleaned[df_cleaned["FraudFound_P"]==0]

In [26]:
# sweetviz report for non fraudulent entries, easier to find trends
non_fraud_eda_report = sv.analyze(non_fraudulent_entries, target_feat='FraudFound_P')
non_fraud_eda_report.show_html('reports/non_fraud_EDA.html')

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report reports/non_fraud_EDA.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


In [100]:
fraud_entries = df_cleaned[df_cleaned["FraudFound_P"]==1]

In [28]:
# sweetviz report for fraudulent entries, easier to find trends
fraud_eda_report = sv.analyze(fraud_entries, target_feat='FraudFound_P')
fraud_eda_report.show_html('reports/fraud_EDA.html')

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


Report reports/fraud_EDA.html was generated! NOTEBOOK/COLAB USERS: the web browser MAY not pop up, regardless, the report IS saved in your notebook/colab files.


Investigating distribution of fraud entries given Sex variable and Age of Policy Holder

In [101]:
fraud_entries['Sex'].value_counts(normalize=True)

1.0    0.886241
0.0    0.113759
Name: Sex, dtype: float64

In [107]:
count_fraud = fraud_entries['AgeOfPolicyHolder'].value_counts()
count_total = df_cleaned['AgeOfPolicyHolder'].value_counts()

percentages = count_fraud/count_total*100
print(percentages)

16 to 17     9.687500
18 to 20    13.333333
21 to 25    14.814815
26 to 30     5.383361
31 to 35     6.436617
36 to 40     5.861984
41 to 50     5.091938
51 to 65     5.028736
over 65      5.905512
Name: AgeOfPolicyHolder, dtype: float64


Age categories with the most fraudulent entries (normalized in the previous output):
- 16 to 17
- 18 to 20
- 21 to 25

Saving the datasets

In [81]:
# saving the dataset as cleaned dataset
df_cleaned.to_csv("data/full_data_cleaned.csv", index=False)
df_cleaned.shape

(15420, 27)

In [82]:
# saving a copy that distinguished fraudulent data
fraud_entries.to_csv("data/fraudulent_data_cleaned.csv", index=False)
fraud_entries.shape

(923, 27)

In [83]:
# saving a copy for non fraudulent data
non_fraudulent_entries.to_csv("data/non_fraudulent_data_cleaned.csv", index=False)
non_fraudulent_entries.shape

(14497, 27)